# Busca no Espaço de Configuração - Hiperparameter Tunning

# Índice:
   * [Descrição](#description)
      * [Tipos de Busca](#searchspace)
      * [Objetivos](#goals)
   * [Imports](#imports)
   * [Extração e Transformação](#extraction)
   * [Parâmetros Alvos da Busca](#parameters)
      * [Introdução a Modelos de Árvores de Decisão](#decisiontree)
      * [Parâmetros do SKLearn](#sklearnparams)
   * [Busca em Grade](#grid)      
      * [Treinamento](#training) 
      * [Validação](#validation)
      * [Log no MLFlow](#mlflow) 
         * [Métricas e Parâmetros](#logmetrics)
         * [Artefato com os Resultados](#logartifact)
   * [Busca Aleatorizada](#randomized)
      * [Treinamento](#randomizedTraining)
      * [Validação](#randomizedValidation)
      * [Log no MLFlow](#randomizedMLFlow)

## Descrição <a class="anchor" id="description"></a>

Esse notebook implementa a busca por híperparâmetros ótimos para o modelo de Árvore de Decisão. Esse modelo foi escolhido por ter um espaço de parâmetros com múltiplas dimensões e por possuir métodos que permitem analisar a tomada de decisão do modelo treinado.

### Tipos de Busca <a class="anchor" id="searchspace"></a>

São implementadas duas estratégias de busca nos híper-parâmetros: a busca randomizada e a busca em grade.

Na busca randomizada são passados uma distribuição de probabilididades para cada parâmetro e o número de iterações. O algoritmo irá gerar valores aleatórios de acordo com a distribuição para cada iteração.

Na busca em grade são passados um conjunto de valores para cada parâmetro e o algoritmo executará todas as combinações. 

Em ambos os casos, são passadas métricas de desempenho e uma delas deve ser determinada como o _score_ a ser utilizado na seleção do melhor modelo. Por motivos de otimização, a biblioteca _Sklearn_ utiliza funções de métricas que são maiores quanto melhor o modelo. Com isso as funções de erro absoluto (funções as quais um valor menor indica um modelo melhor) são multiplicadas por -1, para que cresçam conforme o desempenho do modelo melhora.

### Objetivos  <a class="anchor" id="goals"></a>

* desenvolver funções para a busca nos híperparâmetros (incluindo a validação e log no MLFlow),
* realizar a análise do desempenho do modelo no espaço dos parâmetros,
* entender como o modelo de arvore de decisão se aplica aos dados do problema

## Imports  <a class="anchor" id="imports"></a>

In [32]:
from time import time
import os
import sys
from tabnanny import verbose
import pandas as pd
import numpy as np
import yaml
from itertools import chain, combinations
import datetime
import tempfile

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.utils import shuffle

import mlflow
import mlflow.sklearn
from mlflow import artifacts
from mlflow.tracking import MlflowClient
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID

import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline


In [ ]:
#%pip install boto3

## Extração e Transformação <a class="anchor" id="extraction"></a>

A busca nos híper-parâmetros é feita no conjunto de treino, e o algoritmo de validação cruzada separa uma porção desse conjunto a cada iteração para usar como teste. Ao final, se o objetivo for produzir um modelo treinado, é utilizado um conjunto de validação para aferir o desempenho do melhor estimador encontrado na busca.

Abaixo são separados os conjuntos de treino e validação

In [2]:
def read_data(url):
    return pd.read_csv(os.path.abspath(url),index_col=0)

In [3]:
df = read_data("../extracao/datanov2.csv")

In [4]:
test_split = 0.3
random_state = 10

In [5]:
def getXy(df):
    r_state = 15
    df = shuffle(df, random_state=r_state)
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X, y

X, y = getXy(df)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=random_state
)


## Parâmetros Alvos da Busca<a class="anchor" id="parameters"></a>

### Introdução a Modelos de Árvores de Decisão <a class="anchor" id="decisiontree"></a>

Uma breve descrição do modelo de árvore de decisão é necessária para proceder na busca por híper-parâmetros, com o intuito de se entender melhor os argumentos os quais se deseja maximizar.

Árvores de decisão são grafos acíclicos que podem ser usados para realizar decisões. Em cada nó interno da árvore possui um índice de uma das características e um limiar. Um input que seja analisado nesse nó seguirá na subárvore à esquerda desse nó se o valor da característica indexada no nó for menor que o limiar, e seguirá na subárvore da direita caso contrário. Nas folhas é feita a previsão, que é constante para todos os nós que terminarem de percorrer a árvore naquele nó. Dessa forma, uma árvore é uma aproximação  constante em trechos do domínio.

### Parâmetros do SKLearn<a class="anchor" id="sklearnparams"></a>:
   * _max_depth_ : limita a altura máxima da árvore. Conforme a altura cresce, o modelo fica mais complexo, isso pode aumentar a performace, mas também pode levar a _over fitting_.
   * _max_features_ : Determina o número máximo de características a serem utilizadas nos nós internos da árvore para dividir as amostras.
   * _min_samples_split_ : número mínimo de amostras para dividir uma folha, transformando-a em um nó interno com duas folhas.
   * _min_samples_leaf_ : número mínimo de amostras permitidas em uma folha.
   
O parâmetro _max_depth_ é o príncipal parâmetro para regular o desempenho e o _over fitting_, mas note que os dois últimos parâmetros podem também ter influência na altura final da árvore, se os valores forem sufientemente altos para o conjunto de dados.



In [19]:
param_grid = {
    "max_depth": range( 7, 12, 1),
    "min_samples_split":  range( 10, 250, 30),
    "min_samples_leaf":  range( 5, 80, 10),
    "n_estimators" :  range( 1, 200, 25),
    "max_leaf_nodes":  range( 250, 1000, 100),
    "random_state": [random_state]
}

## Métricas<a class="anchor" id="metrics"></a>

As métricas utilizadas são as mesmas dos últimos modelos: raíz do erro quadrático médio, erro absoluto médio e coeficiente de determinação (r2). Essa última é utilizada como _score_ pela busca em grade para determinar o melhor modelo, que será retreinado utilizando todo conjunto de treino, após a etapa de busca dos melhores parâmetros. Esse retreinamento (refit) permite tanto a validação do modelo na parcela dos dados não utilizada na busca quanto o log do modelo no MLFlow, que poderá ser baixado para uma análise posterior ou mesmo para um possível _deploy_.


In [10]:
_scoring = [
    "r2",
    "neg_mean_absolute_error",
    "neg_root_mean_squared_error",
    ]

## Treinamento <a class="anchor" id="training"></a>

Nesta etapa é treinado o modelo de Árvore de Decisão. O log do modelo no MLFlow é feito logo após o treinamento, por conta de uma limitação da API.

In [24]:
def connectMLFlow(MLFlowAddr):
    client = MlflowClient(tracking_uri=MLFlowAddr)
    mlflow.set_tracking_uri(MLFlowAddr)
    return client

In [12]:
client = connectMLFlow("http://172.27.0.1:5000")

In [13]:
experiment_name = "ModelsSearch"
try:
    experiment_id = client.create_experiment(experiment_name)
except:
    experiment_id = client.get_experiment_by_name(experiment_name).experiment_id
    
experiment = mlflow.set_experiment(experiment_name)


In [14]:
skmodel = RandomForestRegressor(random_state=random_state)

In [22]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [23]:
def logCVResultsCSV(cv_results, run_id, mlclient ):

    tempdir = tempfile.TemporaryDirectory(suffix=None, prefix=None, dir=None)
    tmpname = tempdir.name
    
    filename = "cv_results.csv"
    csv = os.path.join(tmpname, filename)
    
    pd.DataFrame(cv_results).to_csv(csv, index=False)
    
    mlclient.log_artifact(run_id,csv, "cv_results")
    tempdir.cleanup()


In [16]:
run_name = "RandomForest RandomSearch"

In [17]:
n_iter = 500
randSearchModel = RandomizedSearchCV(
    skmodel, param_grid, scoring=_scoring, refit="neg_mean_absolute_error", verbose=2, n_iter=n_iter, random_state=random_state
)

In [18]:
rand_start = time()
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    randSearchModel.fit(X_train, y_train)
    mlflow.sklearn.log_model(randSearchModel.best_estimator_,"model/")
    mlflow.end_run()
rand_end = time()
rand_run_time = rand_end - rand_start

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV] END max_depth=10, max_leaf_nodes=350, min_samples_leaf=35, min_samples_split=10, n_estimators=176, random_state=10; total time=   1.6s
[CV] END max_depth=10, max_leaf_nodes=350, min_samples_leaf=35, min_samples_split=10, n_estimators=176, random_state=10; total time=   1.6s
[CV] END max_depth=10, max_leaf_nodes=350, min_samples_leaf=35, min_samples_split=10, n_estimators=176, random_state=10; total time=   1.5s
[CV] END max_depth=10, max_leaf_nodes=350, min_samples_leaf=35, min_samples_split=10, n_estimators=176, random_state=10; total time=   1.5s
[CV] END max_depth=10, max_leaf_nodes=350, min_samples_leaf=35, min_samples_split=10, n_estimators=176, random_state=10; total time=   1.5s
[CV] END max_depth=8, max_leaf_nodes=650, min_samples_leaf=65, min_samples_split=160, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=8, max_leaf_nodes=650, min_samples_leaf=65, min_samples_split=160, n_estimator

[CV] END max_depth=9, max_leaf_nodes=250, min_samples_leaf=35, min_samples_split=190, n_estimators=76, random_state=10; total time=   0.7s
[CV] END max_depth=8, max_leaf_nodes=550, min_samples_leaf=15, min_samples_split=130, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=550, min_samples_leaf=15, min_samples_split=130, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=550, min_samples_leaf=15, min_samples_split=130, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=550, min_samples_leaf=15, min_samples_split=130, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=550, min_samples_leaf=15, min_samples_split=130, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=350, min_samples_leaf=55, min_samples_split=220, n_estimators=101, random_state=10; total time=   0.7s
[CV] END max_depth=8, max_

[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=75, min_samples_split=220, n_estimators=126, random_state=10; total time=   0.8s
[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=75, min_samples_split=220, n_estimators=126, random_state=10; total time=   0.8s
[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=75, min_samples_split=220, n_estimators=126, random_state=10; total time=   0.9s
[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=75, min_samples_split=220, n_estimators=126, random_state=10; total time=   0.8s
[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=75, min_samples_split=220, n_estimators=126, random_state=10; total time=   0.9s
[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=65, min_samples_split=220, n_estimators=1, random_state=10; total time=   0.0s
[CV] END max_depth=7, max_leaf_nodes=550, min_samples_leaf=65, min_samples_split=220, n_estimators=1, random_state=10; total time=   0.0s
[CV] END max_depth=7, ma

[CV] END max_depth=11, max_leaf_nodes=850, min_samples_leaf=65, min_samples_split=220, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=7, max_leaf_nodes=250, min_samples_leaf=35, min_samples_split=70, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=7, max_leaf_nodes=250, min_samples_leaf=35, min_samples_split=70, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=7, max_leaf_nodes=250, min_samples_leaf=35, min_samples_split=70, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=7, max_leaf_nodes=250, min_samples_leaf=35, min_samples_split=70, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=7, max_leaf_nodes=250, min_samples_leaf=35, min_samples_split=70, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=8, max_leaf_nodes=450, min_samples_leaf=5, min_samples_split=10, n_estimators=126, random_state=10; total time=   1.1s
[CV] END max_depth=8, max_leaf_n

[CV] END max_depth=8, max_leaf_nodes=950, min_samples_leaf=25, min_samples_split=100, n_estimators=26, random_state=10; total time=   0.2s
[CV] END max_depth=11, max_leaf_nodes=250, min_samples_leaf=55, min_samples_split=70, n_estimators=126, random_state=10; total time=   1.0s
[CV] END max_depth=11, max_leaf_nodes=250, min_samples_leaf=55, min_samples_split=70, n_estimators=126, random_state=10; total time=   1.0s
[CV] END max_depth=11, max_leaf_nodes=250, min_samples_leaf=55, min_samples_split=70, n_estimators=126, random_state=10; total time=   1.1s
[CV] END max_depth=11, max_leaf_nodes=250, min_samples_leaf=55, min_samples_split=70, n_estimators=126, random_state=10; total time=   1.1s
[CV] END max_depth=11, max_leaf_nodes=250, min_samples_leaf=55, min_samples_split=70, n_estimators=126, random_state=10; total time=   1.0s
[CV] END max_depth=11, max_leaf_nodes=450, min_samples_leaf=45, min_samples_split=100, n_estimators=101, random_state=10; total time=   0.9s
[CV] END max_depth=1

...

[CV] END max_depth=11, max_leaf_nodes=950, min_samples_leaf=35, min_samples_split=190, n_estimators=101, random_state=10; total time=   1.0s
[CV] END max_depth=9, max_leaf_nodes=650, min_samples_leaf=5, min_samples_split=10, n_estimators=26, random_state=10; total time=   0.3s
[CV] END max_depth=9, max_leaf_nodes=650, min_samples_leaf=5, min_samples_split=10, n_estimators=26, random_state=10; total time=   0.3s
[CV] END max_depth=9, max_leaf_nodes=650, min_samples_leaf=5, min_samples_split=10, n_estimators=26, random_state=10; total time=   0.3s
[CV] END max_depth=9, max_leaf_nodes=650, min_samples_leaf=5, min_samples_split=10, n_estimators=26, random_state=10; total time=   0.4s
[CV] END max_depth=9, max_leaf_nodes=650, min_samples_leaf=5, min_samples_split=10, n_estimators=26, random_state=10; total time=   0.3s
[CV] END max_depth=9, max_leaf_nodes=850, min_samples_leaf=45, min_samples_split=130, n_estimators=151, random_state=10; total time=   1.5s
[CV] END max_depth=9, max_leaf_nod

[CV] END max_depth=8, max_leaf_nodes=250, min_samples_leaf=25, min_samples_split=10, n_estimators=76, random_state=10; total time=   0.7s
[CV] END max_depth=8, max_leaf_nodes=250, min_samples_leaf=25, min_samples_split=10, n_estimators=76, random_state=10; total time=   0.7s
[CV] END max_depth=7, max_leaf_nodes=750, min_samples_leaf=75, min_samples_split=10, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=7, max_leaf_nodes=750, min_samples_leaf=75, min_samples_split=10, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=7, max_leaf_nodes=750, min_samples_leaf=75, min_samples_split=10, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=7, max_leaf_nodes=750, min_samples_leaf=75, min_samples_split=10, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=7, max_leaf_nodes=750, min_samples_leaf=75, min_samples_split=10, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=8, max_leaf_nod

[CV] END max_depth=11, max_leaf_nodes=350, min_samples_leaf=45, min_samples_split=40, n_estimators=176, random_state=10; total time=   1.5s
[CV] END max_depth=11, max_leaf_nodes=350, min_samples_leaf=45, min_samples_split=40, n_estimators=176, random_state=10; total time=   1.6s
[CV] END max_depth=8, max_leaf_nodes=850, min_samples_leaf=35, min_samples_split=70, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=850, min_samples_leaf=35, min_samples_split=70, n_estimators=76, random_state=10; total time=   0.7s
[CV] END max_depth=8, max_leaf_nodes=850, min_samples_leaf=35, min_samples_split=70, n_estimators=76, random_state=10; total time=   0.7s
[CV] END max_depth=8, max_leaf_nodes=850, min_samples_leaf=35, min_samples_split=70, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=8, max_leaf_nodes=850, min_samples_leaf=35, min_samples_split=70, n_estimators=76, random_state=10; total time=   0.6s
[CV] END max_depth=11, max_lea

[CV] END max_depth=8, max_leaf_nodes=650, min_samples_leaf=65, min_samples_split=220, n_estimators=176, random_state=10; total time=   1.3s
[CV] END max_depth=8, max_leaf_nodes=650, min_samples_leaf=65, min_samples_split=220, n_estimators=176, random_state=10; total time=   1.3s
[CV] END max_depth=8, max_leaf_nodes=650, min_samples_leaf=65, min_samples_split=220, n_estimators=176, random_state=10; total time=   1.3s
[CV] END max_depth=10, max_leaf_nodes=450, min_samples_leaf=45, min_samples_split=130, n_estimators=51, random_state=10; total time=   0.5s
[CV] END max_depth=10, max_leaf_nodes=450, min_samples_leaf=45, min_samples_split=130, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=10, max_leaf_nodes=450, min_samples_leaf=45, min_samples_split=130, n_estimators=51, random_state=10; total time=   0.4s
[CV] END max_depth=10, max_leaf_nodes=450, min_samples_leaf=45, min_samples_split=130, n_estimators=51, random_state=10; total time=   0.5s
[CV] END max_depth=1

/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Validação<a class="anchor" id="randomizedValidation"></a>

In [21]:
randPredictVal = randSearchModel.predict(X_val)
randPredictTrain = randSearchModel.predict(X_train)

rmse, mae, r2 = eval_metrics(y_train, randPredictTrain)
print (f"Desempenho no conjunto de Treino\nrmse:{rmse}    mae:{mae}    r2:{r2}")

rmse, mae, r2 = eval_metrics(y_val, randPredictVal)
print (f"\nDesempenho no conjunto de Testes (Validação)\nrmse:{rmse}    mae:{mae}    r2:{r2}")
#print ("\nTempo de busca: {:.3f}s".format(rand_end-rand_start))

NameError: name 'eval_metrics' is not defined

Vemos que o modelo encontrado apresentou um desempenho similar do que o da busca por grade, mesmo com menos tempo de treino. Abaixo estão os valores dos parâmetros do modelo de melhor desempenho na busca e as previsões feitas pelo modelo comparadas com os valores verdadeiros:

In [96]:
randSearchModel.best_params_

{'random_state': 10,
 'n_estimators': 1024,
 'min_samples_split': 8,
 'min_samples_leaf': 8,
 'max_leaf_nodes': 2048,
 'max_depth': 11}

In [97]:
randSearchModel.best_estimator_.feature_importances_

array([0.06508111, 0.04761404, 0.45673176, 0.13724669, 0.08077115,
       0.21255525])

### Log no MLFlow<a class="anchor" id="randomizedMLFlow"></a>

In [104]:
params = "{'max_depth': range( 1, 20, 2),'min_samples_split': [2**x for x in range(12)],'min_samples_leaf': [2**x for x in range(12)],'n_estimators' : [2**x for x in range(12)],'max_leaf_nodes': [2**x for x in range(12)]}"

In [28]:
tags = {"search_type":"RandomSearchCV", 
        "model_random_state":str(random_state),
        "test_size":str(test_split),
        "test_random_state":str(random_state)
       }
tags = [mlflow.entities.RunTag(k,v) for k,v in tags.items()]

search_params = {"param_distributions":str(param_grid),
                 "scoring":str(_scoring),
                 "refit":"neg_mean_absolute_error",
                 "n_iter":str(n_iter),
                 "estimator":"DecisionTreeRegressor",
                "random_state":str(random_state)
                }
search_params = [mlflow.entities.Param(k,v) for k,v in search_params.items()]

metrics = {"rmse":rmse, "mae":mae, "r2":r2}
now = int(time()*1000)
metrics = [mlflow.entities.Metric(k,v,now,1) for k,v in metrics.items()]
client.log_batch(run_id, metrics = metrics, params=search_params,tags = tags)
    
logCVResultsCSV(randSearchModel.cv_results_, run_id, client)


## K-Neighboors

In [35]:
param_grid = {
    "n_neighbors": range( 5, 100, 10),
    "weights":  ["uniform","distance"],
    "leaf_size":  range( 5, 60, 10),
}

_scoring = [
    "r2",
    "neg_mean_absolute_error",
    "neg_root_mean_squared_error",
    ]

skmodel = KNeighborsRegressor()
run_name = "KNeighbors RandomSearch"
n_iter = 500
randSearchModel = RandomizedSearchCV(
    skmodel, param_grid, scoring=_scoring, refit="neg_mean_absolute_error", verbose=2, n_iter=n_iter, random_state=random_state
)



In [36]:
start = time()
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    randSearchModel.fit(X_train, y_train)
    mlflow.sklearn.log_model(randSearchModel.best_estimator_,"model/")
    mlflow.end_run()
end = time()
run_time = rand_end - rand_start

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 120 is smaller than n_iter=500. Running 120 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV] END ........leaf_size=5, n_neighbors=5, weights=uniform; total time=   0.1s
[CV] END ........leaf_size=5, n_neighbors=5, weights=uniform; total time=   0.0s
[CV] END ........leaf_size=5, n_neighbors=5, weights=uniform; total time=   0.0s
[CV] END ........leaf_size=5, n_neighbors=5, weights=uniform; total time=   0.0s
[CV] END ........leaf_size=5, n_neighbors=5, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=5, weights=distance; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=5, weights=distance; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=5, weights=distance; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=5, weights=distance; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=5, weights=distance; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=15, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=5, n_neighbors=15, w

[CV] END ......leaf_size=15, n_neighbors=15, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=15, weights=uniform; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=15, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=15, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=15, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=15, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=15, weights=distance; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END .....leaf_size=15, 

[CV] END ......leaf_size=25, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=25, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=25, n_neighbors=25, weights=uniform; total time=   0.0s
[CV] END .....leaf_size=25, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END .....leaf_size=25, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END .....leaf_size=25, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END .....leaf_size=25, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END .....leaf_size=25, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END ......leaf_size=25, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=25, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=25, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=25, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=25,

[CV] END .....leaf_size=35, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END .....leaf_size=35, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END .....leaf_size=35, n_neighbors=25, weights=distance; total time=   0.0s
[CV] END ......leaf_size=35, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=35, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=35, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=35, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=35, n_neighbors=35, weights=uniform; total time=   0.0s
[CV] END .....leaf_size=35, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END .....leaf_size=35, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END .....leaf_size=35, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END .....leaf_size=35, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END .....leaf_size=35, 

[CV] END .....leaf_size=45, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END .....leaf_size=45, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END .....leaf_size=45, n_neighbors=35, weights=distance; total time=   0.0s
[CV] END ......leaf_size=45, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=45, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=45, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=45, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=45, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END .....leaf_size=45, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=45, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=45, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=45, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=45, 

[CV] END ......leaf_size=55, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=55, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=55, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=55, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=55, n_neighbors=45, weights=uniform; total time=   0.0s
[CV] END .....leaf_size=55, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=55, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=55, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=55, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END .....leaf_size=55, n_neighbors=45, weights=distance; total time=   0.0s
[CV] END ......leaf_size=55, n_neighbors=55, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=55, n_neighbors=55, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=55,

In [37]:
randPredictVal = randSearchModel.predict(X_val)
randPredictTrain = randSearchModel.predict(X_train)

rmse, mae, r2 = eval_metrics(y_train, randPredictTrain)
print (f"Desempenho no conjunto de Treino\nrmse:{rmse}    mae:{mae}    r2:{r2}")

rmse, mae, r2 = eval_metrics(y_val, randPredictVal)
print (f"\nDesempenho no conjunto de Testes (Validação)\nrmse:{rmse}    mae:{mae}    r2:{r2}")


Desempenho no conjunto de Treino
rmse:13.922295753001054    mae:11.055050917316347    r2:0.1977384856819593

Desempenho no conjunto de Testes (Validação)
rmse:13.895091199526519    mae:10.966711800130652    r2:0.16053562027562307


In [40]:
tags = {"search_type":"RandomSearchCV", 
        "test_size":str(test_split),
        "test_random_state":str(random_state)
       }
tags = [mlflow.entities.RunTag(k,v) for k,v in tags.items()]

search_params = {"param_distributions":str(param_grid),
                 "scoring":str(_scoring),
                 "refit":"neg_mean_absolute_error",
                 "n_iter":str(n_iter),
                 "estimator":"KNeighborsRegressor",
                "random_state":str(random_state)
                }
search_params = [mlflow.entities.Param(k,v) for k,v in search_params.items()]

metrics = {"rmse":rmse, "mae":mae, "r2":r2}
now = int(time()*1000)
metrics = [mlflow.entities.Metric(k,v,now,1) for k,v in metrics.items()]
client.log_batch(run_id, metrics = metrics, params=search_params,tags = tags)
    
logCVResultsCSV(randSearchModel.cv_results_, run_id, client)


In [44]:
from sklearn.neural_network import MLPRegressor

In [59]:
param_grid = {
    "hidden_layer_sizes": [(10,15),(20,15),(30,15),(10,5),(20,5),(30,5),(10,25),(20,25),(30,25)],
    "alpha":  10. ** -np.arange( 1, 7),
}

skmodel = MLPRegressor(random_state=random_state,solver="lbfgs", max_iter = 1500)
run_name = "NeuralNetwork RandomSearch"

randSearchModel = GridSearchCV(
    skmodel, param_grid, scoring=_scoring, refit="neg_mean_absolute_error", verbose=2
)




In [60]:
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    randSearchModel.fit(X_train, y_train)
    mlflow.sklearn.log_model(randSearchModel.best_estimator_,"model/")
    mlflow.end_run()


Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 15); total time=   5.8s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 15); total time=   3.2s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 15); total time=   1.8s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 15); total time=   1.1s
[CV] END .............alpha=0.1, hidden_layer_sizes=(10, 15); total time=   3.3s
[CV] END .............alpha=0.1, hidden_layer_sizes=(20, 15); total time=   5.2s
[CV] END .............alpha=0.1, hidden_layer_sizes=(20, 15); total time=   9.8s
[CV] END .............alpha=0.1, hidden_layer_sizes=(20, 15); total time=  11.1s
[CV] END .............alpha=0.1, hidden_layer_sizes=(20, 15); total time=   8.0s
[CV] END .............alpha=0.1, hidden_layer_sizes=(20, 15); total time=   5.4s
[CV] END .............alpha=0.1, hidden_layer_sizes=(30, 15); total time=   4.1s
[CV] END .............alpha=0.1, hidden_layer_s

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END .............alpha=0.1, hidden_layer_sizes=(30, 15); total time=  12.9s


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END .............alpha=0.1, hidden_layer_sizes=(30, 15); total time=  13.5s
[CV] END .............alpha=0.1, hidden_layer_sizes=(30, 15); total time=   5.5s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(10, 5); total time=   1.8s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(10, 5); total time=   5.0s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(10, 5); total time=   4.6s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(10, 5); total time=   2.0s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(10, 5); total time=   2.3s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(20, 5); total time=   7.4s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(20, 5); total time=   2.1s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(20, 5); total time=   3.1s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(20, 5); total time=   2.9s
[CV] END ..............alpha=0.1, hidden_layer_sizes=(20, 5); total time=   2.5s
[CV] END ..............alpha

...

[CV] END ...........alpha=1e-06, hidden_layer_sizes=(20, 25); total time=  15.5s
[CV] END ...........alpha=1e-06, hidden_layer_sizes=(20, 25); total time=  14.5s
[CV] END ...........alpha=1e-06, hidden_layer_sizes=(20, 25); total time=   4.4s
[CV] END ...........alpha=1e-06, hidden_layer_sizes=(20, 25); total time=   9.1s


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END ...........alpha=1e-06, hidden_layer_sizes=(30, 25); total time=  15.9s
[CV] END ...........alpha=1e-06, hidden_layer_sizes=(30, 25); total time=  10.6s


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END ...........alpha=1e-06, hidden_layer_sizes=(30, 25); total time=  15.3s
[CV] END ...........alpha=1e-06, hidden_layer_sizes=(30, 25); total time=  11.2s
[CV] END ...........alpha=1e-06, hidden_layer_sizes=(30, 25); total time=  11.9s


In [61]:
randPredictVal = randSearchModel.predict(X_val)
randPredictTrain = randSearchModel.predict(X_train)

rmse, mae, r2 = eval_metrics(y_train, randPredictTrain)
print (f"Desempenho no conjunto de Treino\nrmse:{rmse}    mae:{mae}    r2:{r2}")

rmse, mae, r2 = eval_metrics(y_val, randPredictVal)
print (f"\nDesempenho no conjunto de Testes (Validação)\nrmse:{rmse}    mae:{mae}    r2:{r2}")


Desempenho no conjunto de Treino
rmse:13.673779159905575    mae:10.799806986782768    r2:0.22612401223613154

Desempenho no conjunto de Testes (Validação)
rmse:13.660030134154024    mae:10.709067016458425    r2:0.18869755564903423


In [62]:
randSearchModel.best_params_

{'alpha': 0.01, 'hidden_layer_sizes': (20, 15)}